https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/qdrant/QA_with_Langchain_Qdrant_and_OpenAI.ipynb

MIT License

Copyright (c) 2025 OpenAI

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# 使用圖形資料庫進行檢索增強生成（RAG）

本教學筆記展示如何將大型語言模型（LLM）與圖形資料庫 Qdrant 結合，來實現 Retrieval Augmented Generation（RAG）。

---

## 為什麼使用 RAG？

如果你希望 LLM 根據你自己的內容或知識庫產生回答，而不是在 prompt 中塞入大量上下文，可以改為**從資料庫中擷取相關資訊**，並以此來生成回應。

這樣做可以達到以下目的：

* ✅ 減少模型幻覺（hallucination）
* ✅ 為使用者提供即時且相關的資訊
* ✅ 利用你自己的內容與知識庫

---

## 為什麼使用圖形資料庫？

如果你的資料特別重視資料點之間的**關係**，或你希望能探索這些關聯性，那麼圖形資料庫會比傳統關聯式資料庫更合適。

圖形資料庫特別擅長以下場景：

* 🔍 深層層級結構的導航
* 🧠 發現項目之間隱藏的連結
* 🕸️ 探索資料點之間的關係

---

## 應用場景

向量資料庫在以下應用中特別有價值：

* 推薦系統
* 社群網路關係分析
* 客戶行為或資料點關聯分析

使用向量資料庫進行 RAG 的實際應用範例包括：

* 🛍️ 產品推薦聊天機器人
* 🤖 AI 增強型 CRM 系統
* 📊 使用自然語言分析使用者行為的工具

---

## 本筆記內容

在本教學中，我們將建立一個 **COVID知識問答機器人**

* 使用上一節準備好的 Qdrant Vector Database 與寫入的 COVID-QA collection 作為知識庫
* 透過 OpenAI embedding model 將使用者輸入的問題產生文字嵌入，使用相同的 `text-embedding-3-large` model 產生的嵌入
* 使用問題的 embedding 進入 Qdrant 做 query_point，找出最接近的 vector，以及對應的 COVID-QA 問答內容
* 將問答內容提供給 LLM，讓 LLM 根據知識庫的問答內容，基於資訊回答問題

👉 [https://www.kaggle.com/datasets/xhlulu/covidqa/data?select=community.csv](https://www.kaggle.com/datasets/xhlulu/covidqa/data?select=community.csv)

---

## 環境設定

我們將從安裝與匯入必要的函式庫開始。

請確保你已經擁有一個 [OpenAI 帳號](https://platform.openai.com/)，並準備好你的 OpenAI API 金鑰。


## 📦 安裝所需套件

In [4]:
! pip install openai qdrant_client tenacity

### 🔐 設定 Azure OpenAI 的 API 金鑰與端點

```python
os.environ["AZURE_OPENAI_API_KEY"]="" # 填上 api key
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview" # 替換成你的 API 版本
os.environ["OPENAI_MODEL"]="text-embedding-3-large"
```

In [17]:
import os

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="gpt-4.1-mini"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

from openai import AzureOpenAI
openai_client = AzureOpenAI()

AZURE_OPENAI_API_KEY is ready


### 建立 qdrant 連線並測試取得 collection

In [7]:
import qdrant_client

client = qdrant_client.QdrantClient(
    host="qdrant",
    prefer_grpc=True,
)

client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='Articles')])

## 🔄 RAG 查詢流程說明

1. **使用者輸入問題（自然語言）**
   例如：「COVID 的全名是什麼？」

2. **將問題轉成向量**
   使用 OpenAI 的 `text-embedding-3-large` 或類似模型轉換問題為向量。

3. **使用向量查詢 Qdrant**
   Qdrant 使用近似最近鄰搜尋（ANN）找出相似的文檔或段落。

4. **取得最相關的 context（上下文）**
   根據 Qdrant 回傳的內容（例如 top-k 筆記錄），你可以拿到與問題相關的段落。

5. **將 context 與原始問題組合成 prompt**
   將 query + context 組合起來，如：

   ```text
   根據以下資料回答問題：
   ===
   [段落1]
   [段落2]
   ===
   問題：COVID 的全名是什麼？
   ```

6. **送出給 GPT 模型進行生成回答**
   傳送給 OpenAI 的 GPT-4 / GPT-3.5 模型生成最終回覆。

---

## 📊 架構流程圖（文字版）

```
┌──────────────┐       ┌─────────────────────┐
│ User Input   │       │ Embedding Model     │
│ (e.g., Query)├──────▶│ (OpenAI Embeddings) │
└──────────────┘       └────────────┬────────┘
                                    │
                                    ▼
                            ┌───────────────┐
                            │ Vector Query  │
                            │ to Qdrant DB  │
                            └──────┬────────┘
                                   ▼
                        ┌────────────────────┐
                        │ Retrieved Contexts │
                        └────────┬───────────┘
                                 ▼
                   ┌────────────────────────────┐
                   │ Prompt Construction Module │
                   │ (Query + Top-K Contexts)   │
                   └────────┬───────────────────┘
                            ▼
                    ┌────────────────────┐
                    │ OpenAI Chat Model  │
                    │ (GPT-4.1 / GPT-4)  │
                    └────────┬───────────┘
                             ▼
                    ┌────────────────────┐
                    │ Final Answer       │
                    └────────────────────┘
```

## 🔄 RAG 查詢流程說明

1. **使用者輸入問題（自然語言）**
   例如：「COVID 的全名是什麼？」

2. **將問題轉成向量**
   使用 OpenAI 的 `text-embedding-3-large` 或類似模型轉換問題為向量。

3. **使用向量查詢 Qdrant**
   Qdrant 使用近似最近鄰搜尋（ANN）找出相似的文檔或段落。

4. **取得最相關的 context（上下文）**
   根據 Qdrant 回傳的內容（例如 top-k 筆記錄），你可以拿到與問題相關的段落。

5. **將 context 與原始問題組合成 prompt**
   將 query + context 組合起來，如：

   ```text
   根據以下資料回答問題：
   ===
   [段落1]
   [段落2]
   ===
   問題：COVID 的全名是什麼？
   ```

6. **送出給 GPT 模型進行生成回答**
   傳送給 OpenAI 的 GPT-4 / GPT-3.5 模型生成最終回覆。

---

## 📊 架構流程圖（文字版）

```
┌──────────────┐       ┌─────────────────────┐
│ User Input   │       │ Embedding Model     │
│ (e.g., Query)├──────▶│ (OpenAI Embeddings) │
└──────────────┘       └────────────┬────────┘
                                    │
                                    ▼
                            ┌───────────────┐
                            │ Vector Query  │
                            │ to Qdrant DB  │
                            └──────┬────────┘
                                   ▼
                        ┌────────────────────┐
                        │ Retrieved Contexts │
                        └────────┬───────────┘
                                 ▼
                   ┌────────────────────────────┐
                   │ Prompt Construction Module │
                   │ (Query + Top-K Contexts)   │
                   └────────┬───────────────────┘
                            ▼
                    ┌────────────────────┐
                    │ OpenAI Chat Model  │
                    │ (GPT-4.1 / GPT-4)  │
                    └────────┬───────────┘
                             ▼
                    ┌────────────────────┐
                    │ Final Answer       │
                    └────────────────────┘
```

In [8]:
from tenacity import retry, wait_random_exponential, stop_after_attempt

@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def get_embedding(text, model="text-embedding-3-large"):
    res = openai_client.embeddings.create(
        model=model,
        input=[text]
    )
    return res.data[0].embedding

def query_docs(query, collection_name="covid-qa-3-large", model="text-embedding-3-large" , top_k=5):
    query_vect = get_embedding(query, model)
    results = client.query_points(
        collection_name=collection_name,
        query=query_vect,
        limit=5,
        with_payload=True,
        using="title"
    )
    payloads = [point.payload["answer"] for point in results.points]
    return payloads

5. **將 context 與原始問題組合成 prompt**
   將 query + context 組合起來，如：

   ```text
   根據以下資料回答問題：
   ===
   [段落1]
   [段落2]
   ===
   問題：COVID 的全名是什麼？
   ```

In [15]:
@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def generate_answer(query, docs, model="gpt-4.1-mini"):
    context = "\n\n".join(docs)
    prompt = f"""根據以下內容回答問題：

1. 請用繁體中文回答
2. 依照內容產生回答
3. 附上內容原文作為依據，原文保留內容的原始語言
4. 如果內容不包含就回答我不知道

內容：
{context}

問題：
{query}
"""

    res = openai_client.chat.completions.create(
        model=model,  # 或你的 Azure 模型名稱
        messages=[
            {"role": "system", "content": "你是一個 helpful AI 助理"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return res.choices[0].message.content.strip()


### 🔄 RAG 查詢

In [10]:
query = "COVID 的全名是什麼"
docs = query_docs(
    query=query,
    collection_name="covid-qa-3-large",
    model="text-embedding-3-large")

answer = generate_answer(
    query=query, 
    docs=docs, 
    model="gpt-4.1-mini")

print("\n🧠 回答：")
print(answer)


🧠 回答：
COVID 的全名是「Coronavirus Disease 2019」，簡稱 COVID-19。

依據原文：
"WHO announced “COVID-19” as the name of this new disease on 11 February 2020, following guidelines previously developed with the World Organisation for Animal Health (OIE) and the Food and Agriculture Organization of the United Nations (FAO)."


### 🔄 RAG 查詢問題2

In [11]:
query = "新冠病毒潛伏期多久？"
docs = query_docs(
    query=query,
    collection_name="covid-qa-3-large",
    model="text-embedding-3-large")

answer = generate_answer(
    query=query, 
    docs=docs, 
    model="gpt-4.1-mini")

print("\n🧠 回答：")
print(answer)


🧠 回答：
根據提供的內容，關於新冠病毒（SARS-CoV-2）的潛伏期資訊如下：

- 在一段引用中提到：「Because fourteen days have elapsed since their departure from Wuhan—longer than the 95th percentile estimate of the COVID-19 incubation period (Li et al., 2020; Linton et al., 2020)」，意即14天已超過新冠病毒潛伏期的95百分位數估計值。

因此，新冠病毒的潛伏期大約是14天以內，這也是目前普遍接受的隔離觀察天數。

---

內容原文依據：

> "Because fourteen days have elapsed since their departure from Wuhan—longer than the 95th percentile estimate of the COVID-19 incubation period (Li et al., 2020; Linton et al., 2020)—there is very little probability that the five virus-positive asymptomatic individuals will develop symptoms."

---

結論：新冠病毒的潛伏期約為14天以內。


### 🔄 檢查 RAG 取得的 context

* 檢查 context 內容
* Qdrant 有撈出對應的資料作為 context
* LLM 確實有使用 context

In [12]:
print("🔍 查詢結果：")
for d in docs:
    print("-", d, "...")

🔍 查詢結果：
- An early release paper dated 23 March 2020 is now staying that SARS-CoV-2 RNA has been recovered 17 days after both infected and asymptomatic presumed infected passengers left the cabins of the Diamond Pricess cruise ship.

The discovery of virus on surfaces of asymptomatic passengers again demonstrates that viral shedding occurs in this phase in amounts sufficient to contaminate the environment.


  SARS-CoV-2 RNA was identified on a variety of surfaces in cabins of both symptomatic and asymptomatic infected passengers up to 17 days after cabins were vacated on the Diamond Princess but before disinfection procedures had been conducted (Takuya Yamagishi, National Institute of Infectious Diseases, personal communication, 2020). Although these data cannot be used to determine whether transmission occurred from contaminated surfaces, further study of fomite transmission of SARS-CoV-2 aboard cruise ships is warranted.


https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e3.htm 
Public 

---
# 小結: Embedding

1. 使用 Qdrant Vector Database 與寫入的 COVID-QA collection 作為知識庫
1. 透過 OpenAI embedding model 將使用者輸入的問題產生文字嵌入，使用相同的 model 產生嵌入
1. 使用問題的嵌入進入 Qdrant 做 query_point，找出最接近的 vector，以及對應的 COVID-QA 問答內容
1. 將問答內容提供給 LLM，讓 LLM 根據知識庫的問答內容回答問題

---

# 👋👋👋自己動手做看看👋👋👋

嘗試使用不同的資料集 `community_embedded_text_embedding_3_small.csv` 進行 Embedding Search

### 目標：讓 query_results 使用另外一個資料集

1. client.get_collections() 會出現多個 collection
2. query_results 

---

### 提示

1. 需要使用 [2_Embedding_Search_with_Qdrant_and_OpenAI](2_Embedding_Search_with_Qdrant_and_OpenAI.ipynb) 建立的新的 collection，如果沒有可以先回到 2
2. 嘗試使用不同的 model （ex. model="gpt-4.1")，結果會有所不同嗎？
3. 如果 collection 與 model 使用不同的 embedding model，結果跑得出來嗎？`collection_name="covid-qa-3-large" model="text-embedding-3-large"`


In [ ]:
query = "想一個新的問題"
docs = query_docs(
    query=query,
    collection_name="",
    model="")

answer = generate_answer(
    query=query, 
    docs=docs, 
    model="")

print("\n🧠 回答：")
print(answer)

---

# 👋👋👋自己動手做看看👋👋👋

嘗試修改不同的 prompt，可以增加更多條件

```
@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def generate_answer(query, docs, model="gpt-4.1-mini"):
    context = "\n\n".join(docs)
    prompt = f"""根據以下內容回答問題：

1. 請用繁體中文回答
2. 依照內容產生回答
3. 附上內容原文作為依據，原文保留內容的原始語言
4. 如果內容不包含就回答我不知道

內容：
{context}

問題：
{query}
"""

    res = openai_client.chat.completions.create(
        model=model,  # 或你的 Azure 模型名稱
        messages=[
            {"role": "system", "content": "你是一個 helpful AI 助理"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return res.choices[0].message.content.strip()
```

In [ ]:
@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def generate_better_answer(query, docs, model="gpt-4.1-mini"):
    context = "\n\n".join(docs)
    prompt = f"""
    
這邊都可以改，發揮創意

內容：
{context}

問題：
{query}
"""

    res = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "你是一個 helpful AI 助理"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return res.choices[0].message.content.strip()


#### 使用相同問題，看使用不同的 prompt，LLM 的回答有和不同 

1. 可以使用 chatGPT 產生 prompt，再把 prompt 丟進來程式碼中
2. 回答有什麼差異
3. 你修改的 prompt 效果，比原先講師提供的 prompt，哪個更好？
4. 呈上，基於什麼標準讓你覺得哪段 prompt 的效果比較好？


In [ ]:
docs = query_docs(
    query=query,
    collection_name="",
    model="")

answer = generate_answer(
    query=query, 
    docs=docs, 
    model="")

print("\n🧠 回答：")
print(answer)